# [IAPR 2018:][iapr2018] Project API tutorial

[iapr2018]: https://github.com/LTS5/iapr-2018

## General information

You will find two identical setups (A and B) in the project room.
They are composed of the arena, an EV3 robot, a computer and a webcam (connected via USB).
Each computer has the same configuration and is able to automatically connect to a single EV3 robot (via Bluetooth).
Login credentials are written on each computer (post-it).

A dedicated Python environment, called `iapr-project`, is already installed.
All the necessary packages for the project should be available.
You can activate it from a terminal (`Ctrl` + `Alt` + `T`) with the following command:
```
source activate iapr-project
```

Both [PyCharm] and [Jupyter] are installed and can be used for the project.

**Important:**
Do not forget that the computers are shared.
Hence, you should always delete your code before leaving the computer.

[jupyter]: https://jupyter.org/
[pycharm]: https://www.jetbrains.com/pycharm/

## WebcamVideoStream interface
The `WebcamVideoStream` interface provided in the `iapr` API will allow you to capture images of the image arena.
Once instantiated, this interface will have a dedicated thread constantly acquiring the webcam stream.
In order to get the most recent frame, just call the `read()` command.

In [ ]:
from iapr.webcam import WebcamVideoStream
import matplotlib.pyplot as plt
%matplotlib inline

# Create a webcam video stream
#   Note: src=0 links to the USB Webcam of the computers provided for the project
wvs = WebcamVideoStream(src=0)

# Read most recent frame
frame = wvs.read()

# Plot
fig, ax = plt.subplots(1, 1, figsize=(6, 6))
ax.imshow(frame)
ax.axis('off')
plt.show()

Note that if you want to explicitly stop the thread and release the Webcam you can use the `stop()` and `release()` methods from the `WebcamVideoStream` interface.

## Robot control

### SSH connection
First, you need to connect the EV3 via SSH.
Just open a terminal (`Ctrl` + `Alt` + `T`) and type the following command:
```
ssh ev3dev
```
Once connected you will see the following output:
```
             _____     _
   _____   _|___ /  __| | _____   __
  / _ \ \ / / |_ \ / _` |/ _ \ \ / /
 |  __/\ V / ___) | (_| |  __/\ V /
  \___| \_/ |____/ \__,_|\___| \_/
    
Debian jessie on LEGO MINDSTORMS EV3!
    
The programs included with the Debian GNU/Linux system are free software;
the exact distribution terms for each program are described in the
individual files in /usr/share/doc/*/copyright.
    
Debian GNU/Linux comes with ABSOLUTELY NO WARRANTY, to the extent
permitted by applicable law.
```

### RPyC server
In order to control remotely the EV3, you need to launch an [RPyC] server.
Once connected via SSH on the EV3 (see [SSH connection](#ssh-connection)) use the following command:
```
./rpyc_server.sh
```
It should output something like
```
INFO:SLAVE/18812:server started on [0.0.0.0]:18812
```

[rpyc]: https://rpyc.readthedocs.io/en/latest/

### Robot interface
The `Robot` interface provided in the `iapr` API will help you to control the EV3 remotely.
This interface provides a few commands to control the EV3:
* `move_forward(distance)` and `move_backward(distance)`:
the `distance` argument has to be a `float` (in centimeters).
* `steer_in_place(angle)`:
the `angle` argument has to be a `float` (in degree).
* `beep(count)`:
the `count` agument has to be an `int` (default=1).

**Important:**
* The `distance` and `angle` arguments you will provide cannot be guaranteed!
They are only approximations of the resulting movement.
The actual movement highly depends on many physical parameters such as friction, sliding and instantaneous power.
Hence, you should monitor the *true* position of the EV3 using the webcam.
* The API is designed to work with the right motor connected to EV3 port A and the left motor connected to EV3 port D.

### Example 1: Create Robot instance and beep

In [ ]:
from iapr.robot import Robot

# Create Robot instance
robot = Robot(hostname='ev3dev.local')

# Beep
robot.beep(count=1)

### Example 2: Move Robot and beep

In [ ]:
# Move forward
distance = 15  # cm
robot.move_forward(distance=distance)
robot.beep(count=1)

# Steer
angle = 90  # deg
robot.steer_in_place(angle=+angle)
robot.steer_in_place(angle=-angle)
robot.beep(count=2)

# Move backward
robot.move_backward(distance=distance)
robot.beep(count=3)

### Example 3: Move Robot and capture images

In [ ]:
# Take initial position image and store in image list
images = list()
images.append(wvs.read())

# Move forward
distance = 30  # cm
robot.move_forward(distance=distance)
images.append(wvs.read())

# Steer
angle = 60  # deg
robot.steer_in_place(angle=angle)
images.append(wvs.read())

# Move backward
robot.move_backward(distance=distance)
images.append(wvs.read())

# Plot images
fig, axes = plt.subplots(2, 2, figsize=(12, 12))
for i, (ax, im) in enumerate(zip(axes.ravel(), images)):
    ax.imshow(im)
    ax.set_title('Position #{:d}'.format(i))
    ax.axis('off')
plt.show()